## NB very simple model for testing

In [1]:
%pip install xgboost
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install seaborn

Active code page: 1252
Note: you may need to restart the kernel to use updated packages.
Active code page: 1252
Note: you may need to restart the kernel to use updated packages.
Active code page: 1252
Note: you may need to restart the kernel to use updated packages.
Active code page: 1252
Note: you may need to restart the kernel to use updated packages.
Active code page: 1252
Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib inline
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import joblib
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error


In [3]:
data = pd.read_csv('final.csv')

In [4]:
data.head(8)

,Date,Time,Player_count
0,9/24/2023,0:00,2711248
1,9/24/2023,0:05,2726267
2,9/24/2023,0:10,2712687
3,9/24/2023,0:15,2745816
4,9/24/2023,0:20,2756843
5,9/24/2023,0:25,2768107
6,9/24/2023,0:30,2777797
7,9/24/2023,0:35,2786147


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5726 entries, 0 to 5725
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          5726 non-null   object
 1   Time          5726 non-null   object
 2   Player_count  5726 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 134.3+ KB


In [6]:
data = data.drop(data[(data["Player_count"]== 0)].index)

In [7]:
data["New_date"] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5544 entries, 0 to 5725
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          5544 non-null   object        
 1   Time          5544 non-null   object        
 2   Player_count  5544 non-null   int64         
 3   New_date      5544 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 216.6+ KB


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5544 entries, 0 to 5725
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          5544 non-null   object        
 1   Time          5544 non-null   object        
 2   Player_count  5544 non-null   int64         
 3   New_date      5544 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 216.6+ KB


### Feature engineering (transform data to be used for the machine learning algo) 

In [10]:

data["New_time"] = pd.to_datetime(data['Time'], format='%H:%M')

In [11]:
data["year"] = data["New_date"].dt.year
data["month"] = data["New_date"].dt.month
data["day"] = data["New_date"].dt.day
data["hour"] = data["New_time"].dt.hour
data["min"] = data["New_time"].dt.minute

In [12]:
data.drop(columns=["Date","Time","New_date","New_time"], inplace=True)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5544 entries, 0 to 5725
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Player_count  5544 non-null   int64
 1   year          5544 non-null   int32
 2   month         5544 non-null   int32
 3   day           5544 non-null   int32
 4   hour          5544 non-null   int32
 5   min           5544 non-null   int32
dtypes: int32(5), int64(1)
memory usage: 194.9 KB


In [14]:
data.head()

,Player_count,year,month,day,hour,min
0,2711248,2023,9,24,0,0
1,2726267,2023,9,24,0,5
2,2712687,2023,9,24,0,10
3,2745816,2023,9,24,0,15
4,2756843,2023,9,24,0,20


### Preapering for training 

### Models and testing

#### xgboost

In [15]:
tss = TimeSeriesSplit(n_splits=5, test_size=11, gap=11)
df = data.sort_index()


fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    FEATURES = ['year', 'month', 'day', 'hour', 'min']
    TARGET = 'Player_count'

    X_train = train[FEATURES].values
    y_train = train[TARGET]

    X_test = test[FEATURES].values
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

[0]	validation_0-rmse:1763742.11169	validation_1-rmse:2696915.29324
[100]	validation_0-rmse:689745.14224	validation_1-rmse:1374556.57847
[200]	validation_0-rmse:323037.53513	validation_1-rmse:872403.38597


c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:04:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[300]	validation_0-rmse:209299.90748	validation_1-rmse:710988.80549
[400]	validation_0-rmse:168957.68920	validation_1-rmse:642595.49688
[500]	validation_0-rmse:148201.63892	validation_1-rmse:613874.76944
[600]	validation_0-rmse:134682.89399	validation_1-rmse:593835.24160
[700]	validation_0-rmse:124218.90459	validation_1-rmse:566047.49520


[800]	validation_0-rmse:114566.71922	validation_1-rmse:508401.83373
[900]	validation_0-rmse:106859.36951	validation_1-rmse:433341.02731
[999]	validation_0-rmse:102373.34472	validation_1-rmse:400768.84643
[0]	validation_0-rmse:1766030.42642	validation_1-rmse:2457419.81400
[100]	validation_0-rmse:691042.94750	validation_1-rmse:1131873.51911
[200]	validation_0-rmse:324253.22010	validation_1-rmse:633071.61101
[300]	validation_0-rmse:210969.03665	validation_1-rmse:472118.72607


c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:04:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


...

[400]	validation_0-rmse:172922.71067	validation_1-rmse:176611.21636
[500]	validation_0-rmse:152606.99643	validation_1-rmse:146546.24310
[600]	validation_0-rmse:138352.82090	validation_1-rmse:112946.01036
[700]	validation_0-rmse:127704.13838	validation_1-rmse:86290.43140
[800]	validation_0-rmse:116788.47958	validation_1-rmse:34877.66239
[836]	validation_0-rmse:113833.21611	validation_1-rmse:62996.43986
[0]	validation_0-rmse:1769999.48714	validation_1-rmse:2499945.28751
[100]	validation_0-rmse:692875.72661	validation_1-rmse:1077008.89431
[200]	validation_0-rmse:325930.01510	validation_1-rmse:557373.55655
[300]	validation_0-rmse:213027.93033	validation_1-rmse:359434.47019


c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:04:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[400]	validation_0-rmse:173481.22531	validation_1-rmse:262611.06403
[500]	validation_0-rmse:152798.40326	validation_1-rmse:222437.56345
[600]	validation_0-rmse:138217.80537	validation_1-rmse:174538.72954
[700]	validation_0-rmse:127515.87192	validation_1-rmse:137743.47235
[800]	validation_0-rmse:117639.96063	validation_1-rmse:70915.13549
[898]	validation_0-rmse:109674.79725	validation_1-rmse:70030.71295
[0]	validation_0-rmse:1771119.59691	validation_1-rmse:2863256.91671
[100]	validation_0-rmse:693092.42676	validation_1-rmse:1272365.87969
[200]	validation_0-rmse:325920.49182	validation_1-rmse:628192.34180
[300]	validation_0-rmse:213207.66699	validation_1-rmse:405486.85822


c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:04:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[400]	validation_0-rmse:173569.21475	validation_1-rmse:303938.64812
[500]	validation_0-rmse:152660.50748	validation_1-rmse:248094.54207
[600]	validation_0-rmse:138189.25776	validation_1-rmse:187113.95045
[700]	validation_0-rmse:127726.56180	validation_1-rmse:138774.66303
[800]	validation_0-rmse:119334.38713	validation_1-rmse:79677.12386
[900]	validation_0-rmse:110919.05665	validation_1-rmse:32226.00823
[919]	validation_0-rmse:109573.66355	validation_1-rmse:45481.03515


In [16]:
print(f'Score across folds {np.mean(scores):0.4f}')
print(f'Fold scores:{scores}')

Score across folds 124054.1949
Fold scores:[400768.84630407306, 127712.31514541242, 28162.610530590508, 50537.6947145643, 13089.50772565569]


Saving model to pkl file for later use


In [17]:
reg.save_model("reg_model.json")
